## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [346]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import matplotlib.pyplot as plt


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def BandpassFilter(signal, bandpass, fs):
    """
    Filter bandpass for unrealistic heart rate values.
    Args:
        signal: numpy array. Signal to process.
        pass_band: tuple. band width for filter in Hz   
        fs: int. Sampling frequency in Hz
    Returns:
        numpy array. Filtered signal
    """
    b, a = scipy.signal.butter(3, bandpass, btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, signal)


def ComputeSpectogram(signal, window, overlap, fs):
    """
    Compute Spectogram
    
    Args:
        signal: numpy array. Signal to process
        window: int. Window in seconds used to collect the data
        overlap: int. Overlap in seconds between consecutive windows   
        fs: int. Sampling frequency in Hz. Default is 125 Hz
    
    Returns:
        spectorgram (FTT)
    """
    nfft_window = fs * window
    noverlap = fs * overlap
    spectrum, freqs, _, _ = plt.specgram(signal, NFFT=nfft_window, Fs=fs, noverlap=noverlap)
    plt.close()
    return spectrum, freqs

def ComputeFreqPeaks(spectrum, freqs, max_freq):
    """
    Compute peaks of the spectogram
    
    Args:
        spectrum: 2D array . Spectogram
        freqs: 1D array. The frequencies corresponding to the rows in Spectrum.

    Optional Arg:    
        max_freq: int. Maximum frequency used to find peaks. Default is 5 Hz
    
    Returns:
        A list of the peaks (frequences) for each time window sorted by dominance
    """

    peaks = []
    
    # For a FFT signal of each time window
    for current_window in range(spectrum.shape[1]):

        # Use Frequencies <= max_freq
        current_spectrum = spectrum[:, current_window]
    
        # A peak is must have a height > 10% of the highest peak
        peak_indices = sp.signal.find_peaks(current_spectrum, height=np.max(np.abs(current_spectrum)*0.1))[0]

        # Sort peaks by descendent dominance
        sorted_indices = np.argsort(current_spectrum[peak_indices])[::-1]

        peaks.append(freqs[peak_indices[sorted_indices]])
        
    return peaks


def ComputePulseRates(signals_peaks, similarity_1=1, similarity_2=0, last_n_estimates=3):
    """
    Compute pulse rate estimation given the peaks of the spectorgrams
    of ppg, accx, accy, and accz signals
    
    Args:
        signal peaks: a dictioray of spectoragms peaks
        similarity_1: similarity to compare ppg and acc dominant frequencies
        similarity_2: similarity to compare dominant ppg frequencies between windows
        last_n_estimates: number of previous windows used to smooth pulse rate

    
    Returns:
        a numpy array of pulse rate estimations per time window (in bmp)
    
    """

    max_ppg_freqs = []
    
    i = 0

    for ppg, accx, accy, accz in zip(signals_peaks["ppg"], signals_peaks["accx"], signals_peaks["accy"], signals_peaks["accz"]):
        
        # Get the dominant ppg frequency for the current window
        current_ppg = list(ppg)
        dominant_ppg = current_ppg[0]
        dominant_accs = [accx[0], accy[0], accz[0]]
        ppg_acc_similarity = [abs(max_acc - dominant_ppg) <= similarity_2 for max_acc in dominant_accs]

        # When the dominant accelerometer frequency is the same as the PPG
        # pick the next strongest PPG frequency if there is another good candidate
        while (sum(ppg_acc_similarity) > 0) and (len(current_ppg) >= 2) :
            current_ppg = current_ppg[1:]
            # print(i, current_ppg)
            dominant_ppg = current_ppg[0]
            ppg_acc_similarity = [abs(dom_acc - dominant_ppg) <= similarity_2 for dom_acc in dominant_accs]
            # print(i, max_ppg )
        
        # If all ppg peaks are similar to the acc peaks chose the most dominant
        # ppg peak as pulse rate estimate
        if len(current_ppg) == 1:
            dominant_ppg = ppg[0]

        # Smooth out the rate estimates
        if i >= last_n_estimates and abs(dominant_ppg - max_ppg_freqs[i-1]) >=  similarity_1:
            dominant_ppg = sum(max_ppg_freqs[-last_n_estimates:]) / last_n_estimates


        max_ppg_freqs.append(dominant_ppg)
        i += 1

    return np.array(max_ppg_freqs) * 60 # Converted to bpm

def ComputeConfidences(pulse_rate_estimates, ppg_spectrum, ppg_freqs, bpm_confidence):
    """
    Compute the confidence in the pulse rate estimations given a bpm confidence interval
    
    Args:
        pulse_rate_estimates: numpy array of pulse rate estimate per time window (bpm)
        ppg_spectrum: ppg spectrum
        ppg_freqs: ppg frequencies 
        bpm_confidence: bmp interval to compute confidence
    Returns:
        numpy array of confidence per time window
    """
    
    confidences = []

    for i in range(len(pulse_rate_estimates)):
        pulse_rate = pulse_rate_estimates[i]
        spectrum = ppg_spectrum[:, i]
        lower_conf_window = pulse_rate - bpm_confidence
        upper_conf_window = pulse_rate + bpm_confidence
        confidence_window = (ppg_freqs >= lower_conf_window) & (ppg_freqs <= upper_conf_window)
        confidence = np.sum(spectrum[confidence_window]) / np.sum(spectrum)
        confidences.append(confidence)
    
    return np.array(confidences)

def ComputeErrors(pulse_rate_estimates, ecg):
    """
    Compute Mean Avergae Error (MAE) in bmp between the pulse rate estimates
    and a reference pulse rate
    
    Args:
        pulse_rate_estimates: numpy array of pulse rate estimation per window (bpm)
        ecg: numpy array of ground thruth pulse rate per window (bpm)
    
    Returns:
        MAE in bpm
    
    """
    length = pulse_rate_estimates.shape[0]
    pulse_rate_estimates = pulse_rate_estimates.reshape(1, length)
    ecg = ecg.reshape(1, length)
    errors = pulse_rate_estimates - ecg
    return errors.tolist()[0]

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Args:
        data_fl: a path of the .mat files that contain signal data
        ref_fl: a path of the .mat files that contain reference data
    
    Returns:
        A 2-tuple of numpy arrays of per-estimate mean absolute error and confidence as a .
    """
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ecg = sp.io.loadmat(ref_fl)['BPM0'].flatten()

    signals = {
        "ppg" : ppg,
        "accx": accx,
        "accy": accy,
        "accz": accz,
    }

    fs = 125
    low_freq = 40/60
    high_freq = 240/60
    window = 8
    overlap = 6
    max_freq = 5
    bpm_confidence = 15
    
    # Compute per window peaks for each signal
    signals_peaks = {}
    bandpass = (low_freq, high_freq)
    for key, signal in signals.items():
        filtered_signal = BandpassFilter(signal, bandpass=bandpass,fs=fs)
        spectrum, freqs = ComputeSpectogram(filtered_signal, window=window, overlap=overlap, fs=fs)
        signals_peaks[key] = ComputeFreqPeaks(spectrum=spectrum, freqs=freqs, max_freq=max_freq)
        # Store spectrum data only for ppg signal
        if key == "ppg":
            ppg_freqs = freqs
            ppg_spectrum = spectrum

    # Compute per window pulse rate estiame in bpm
    pulse_rate_estimates = ComputePulseRates(signals_peaks)
    # Compute per window errors
    errors = ComputeErrors(pulse_rate_estimates, ecg)
    # Compute per window confidence
    confidences = ComputeConfidences(pulse_rate_estimates, ppg_spectrum, ppg_freqs, bpm_confidence)
    #   pulse_rate_estimates, ecg, signals_peaks
    return errors, confidences


In [721]:
Evaluate()

14.901684582495966

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

- **Code Description**

Run the first cell to load all the functions required for the pulse rate estimation algorithm and the second cell to run the evaluation function.

The evaluation functions first loads the Troika data, runs the estimation algorithm and returns the Mean Average Error at 90% confidence.

- **Data Description**

We use the [Troika](https://ieeexplore.ieee.org/document/6905737) dataset. It contains 12 data points. Each of these data point comes with the PPG signal, the 3-axis accelerometer signals, and a reference ecg signal.

- **Algorithn Description**

We take advantage of the following physiologic aspects:
 1. The light emitted by the PPG sensor is absorbed by red blood cells in these capilaries and the photodetector will see the drop in reflected light. When the blood returns to the heart, fewer red blood cells in the wrist absorb the light and the photodetector sees an increase in reflected light. The period of this oscillating waveform is the pulse rate.
 2. The PPG sensor also register blood pressure due to the arm motion during activities like walking and runninng. The accelerometer register the periodic movement of these activities and make it possible to filter them out from the PPG signal.

The algorithm takes the PPG and the 3-axis accelerometer signals as input and outputs the pulse rate estimation for each time window. It follows these steps:
 1. Filter the PPG signal and the 3-axis accelerometer signals to keep the frequencies between the range of 40-240 bpm.
 2. Compute the spectograms for each of the signals.
 3. Estimate the pulse rate use the dominant frequency from the PPG spectogram. However, uf the first dominant ppg frequency is similar to one the dominant frequency of any accelerometer signals, we use the next dominant frequency of the ppg spectogram as an estimation. If all the dominant PPG frequencies are similar to the accelerometer, we pick the dominant PGG frequency.
    
Caveats and common failure mode:
 1. The algorithm doesn't accound for body movements like the fingers movements because they can't be capture by the accelerometer.
 2. The agorithm could perform poorly of periodic body movenment other then running. The training set contains ideed only accelerometer signals of running on a trademill.
 3. The algorithm doesn't account for additional sources of noice like surrounding light and displacement of the PPG sensor on the hand.
 4. The algortim doesn't account for non periodic movement captured by the sensor.

- **Algorithm Performance**

The Mean Absolute Error at 90% availabiliy is 14.9 BPM, which lower than the minimum required MAE (15 BPM) on the training set. This MAE is 7BPM on the test set.

We compute the average absoulte difference between the estimated pulse rate and a reference pulse rate fron an ECG.

-----
### Next Steps
You will now go to **Test Your Algorithm** (back in the Project Classroom) to apply a unit test to confirm that your algorithm met the success criteria. 




<img src="./passed.png" alt="Alternative text" />